<a href="https://colab.research.google.com/github/jihoover77/DS-Unit-2-Linear-Models/blob/master/Linear_Regression2_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [34]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

# Step 1:  Read in, wrangle and feature engineer the dataset

In [35]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def wrangle(filepath):

  # Read New York City apartment rental listing data
  df = pd.read_csv(filepath,
                   parse_dates=['created'],
                   index_col='created')
  
  # Remove the most extreme 1% prices,
  # the most extreme .1% latitudes, &
  # the most extreme .1% longitudes
  df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

  # Drop duplicate column
  df.drop(columns='street_address', inplace=True)

  # Drop high cardinality features
  threshold = 3
  drop_cols = [col for col in df.select_dtypes('object').columns
               if df[col].nunique() > threshold]
  df.drop(columns=drop_cols, inplace=True)

  # Engineer a feature called perks_count
  df['perks_count'] = df[['elevator', 'cats_allowed',
                          'hardwood_floors', 'dogs_allowed', 'doorman', 
                          'dishwasher', 'no_fee',
                          'laundry_in_building', 'fitness_center', 'pre-war',
                          'laundry_in_unit',
                          'roof_deck', 'outdoor_space', 'dining_room',
                          'high_speed_internet',
                          'balcony', 'swimming_pool', 'new_construction',
                          'terrace', 'exclusive',
                          'loft', 'garden_patio', 'wheelchair_access',
                          'common_outdoor_space']].sum(axis=1)
  
  # Engineer a feature called total_rooms
  df['total_rooms'] = df['bathrooms'] + df['bedrooms']

  # Engineer a column that checks for cats or dogs
  df.loc[(df['cats_allowed'] == 1) | (df['dogs_allowed'] == 1), 'cat_or_dogs'] = 1
  df.loc[(df['cats_allowed'] != 1) & (df['dogs_allowed'] != 1), 'cat_or_dogs'] = 0
  
  # Engineer a column that checks for cats and dogs
  df.loc[(df['cats_allowed'] == 1) & (df['dogs_allowed'] == 1), 'cat_and_dogs'] = 1
  df.loc[(df['cats_allowed'] != 1) | (df['dogs_allowed'] != 1), 'cat_and_dogs'] = 0

  # Create a feature column from latitude and longitude to get the distance from
  # Manhattan 
  manhattan_lat = 40.7829
  manhattan_lon = -73.9711
  df['distance_from_Manhattan'] = np.sqrt((manhattan_lat - df['latitude'])**2 + 
                                        (manhattan_lon - df['longitude'])**2)
  
  return df

df = wrangle(DATA_PATH + 'apartments/renthop-nyc.csv')

In [36]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,perks_count,total_rooms,cat_or_dogs,cat_and_dogs,distance_from_Manhattan
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0.0,0.0,0.074139
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3.0,1.0,1.0,0.012594
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2.0,0.0,0.0,0.053734
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2.0,0.0,0.0,0.029199
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5.0,0.0,0.0,0.046612


# Step 2:  Split Data

In [38]:
df.columns

Index(['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price',
       'interest_level', 'elevator', 'cats_allowed', 'hardwood_floors',
       'dogs_allowed', 'doorman', 'dishwasher', 'no_fee',
       'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit',
       'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet',
       'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive',
       'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space',
       'perks_count', 'total_rooms', 'cat_or_dogs', 'cat_and_dogs',
       'distance_from_Manhattan'],
      dtype='object')

In [39]:
target = 'price'
features = ['total_rooms', 'perks_count', 'distance_from_Manhattan']
X = df[features]
y = df[target]

X.shape, y.shape

((48817, 3), (48817,))

In [41]:
cutoff = '2016-06-01'
mask = df.index < cutoff


X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((31844, 3), (31844,), (16973, 3), (16973,))

# Step 3:  Establish Baseline

In [43]:
y_pred = [y_train.mean()] * len(y_train)

print(f'Baseline MAE:  ${mean_absolute_error(y_train, y_pred):,.0f}')

Baseline MAE:  $1,202


# Step 4:  Build Model

In [44]:
# Step 1:  Import predictor class
# Done above

# Step 2:  Instantiate the class
model = LinearRegression()

# Step 3:  Fit the model to training set
model.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Step 5: Check Metrics

In [48]:
y_pred = model.predict(X_test)
print(f'Linear Model Training MAE: ${mean_absolute_error(y_train, model.predict(X_train)):,.0f}')
print(f'Linear Model Testing MAE: ${mean_absolute_error(y_test, model.predict(X_test)):,.0f}')
print(f'Linear Model Training RMSE: ${mean_squared_error(y_train, model.predict(X_train), squared=False):,.0f}')
print(f'Linear Model Testing RMSE: ${mean_squared_error(y_test, model.predict(X_test), squared=False):,.0f}')
print('Linear Model Training R^2: ', model.score(X_train, y_train))
print('Linear Model Testing R^2: ', model.score(X_test, y_test))

Linear Model Training MAE: $845
Linear Model Testing MAE: $853
Linear Model Training RMSE: $1,264
Linear Model Testing RMSE: $1,260
Linear Model Training R^2:  0.4851614249786903
Linear Model Testing R^2:  0.48882433761808775


# Step 6:  Communicate the Results

In [55]:
intercept = round(model.intercept_)
coef_1 = model.coef_[0]
coef_2 = model.coef_[1]
coef_3 = model.coef_[2]

print(f'PRICE = {intercept:,.0f} + ({coef_1:,.0f} * ROOMS) + ({coef_2:,.0f} * PERKS) - ({-coef_3:,.0f} * DISTANCE)')

PRICE = 1,434 + (775 * ROOMS) + (96 * PERKS) - (8,844 * DISTANCE)


### I added three new features to my dataset.  The best score I received was from the mean absolute error $853.  